---

# IT00CJ42: Search and Optimization Algorithms

**Restart the kernel and run all cells** before you turn this problem in, make sure everything runs as expected.

Make sure you fill in any place that says `YOUR CODE HERE`.

---

## Week 4

Solve QAP with tabu search

### Imports and setting seed

In [16]:
import math
import numpy as np
import random

### Solution check
We use the function **check** to implement tests for your solution

In [17]:
def check(expression, message=""):
    if not expression:
        raise AssertionError(message)
    print(message+" : passed.")

## Task 1: Implement neighbors and best_neighbor_swap

In [18]:
class Solution:
    def __init__(self, permutation, flows, distances):
        assert flows.shape == distances.shape, "Flow and distance matrices have to be of the same shape."
        self.permutation = np.array(permutation)
        self.flows = flows
        self.distances = distances
        self.optimal = None

    def __str__(self):
        return str(self.permutation)
    def __repr__(self):
        return str(self.permutation)

    # Load a dat file with a problem instance
    @classmethod
    def from_file(cls,  filename):
        # load in all rows into lists
        dat = [i.strip().split() for i in open(filename).readlines()]
        # remove empty lists
        res = [ele for ele in dat if ele != []]
        # convert strings to integers
        temp = [list(map(int, x)) for x in res]

        # first row: size, optimal value
        info = temp[0]
        size = info[0]
        optimal = info[1]

        # size*rows down is the distance
        distances = np.array(temp[1:size + 1]).reshape(size, size)
        # size*rows after that is the flow
        flows = np.array(temp[size + 1:len(temp)]).reshape(size, size)
        permutation= np.random.permutation(distances.shape[0])
        sol=cls(permutation,flows,distances)
        sol.optimal=optimal
        return sol

    @property
    def objective(self):
          return self.evaluate_objective(self.permutation, self.flows, self.distances)

    @classmethod
    def evaluate_objective(cls, permutation, flows, distances):
        # When doing a swap the objective can be updated with less cost, but we
        # do not care.
        objective = 0.0
        for i in range(flows.shape[0]):
            for j in range(i + 1, flows.shape[1]):
                 objective += flows[i, j] * distances[permutation[i], permutation[j]]

        # The matrices are symmetric.
        return 2*objective
        
    def neighbors(self, tabu=None, threshold=0):
        # TODO: Implement this. The returned value should be a list of tuples
        # (i,j) representing non-tabu swaps.
        neighborhood = []
        for i in range(len(self.permutation)):
            for j in range(i + 1, len(self.permutation)):
                if tabu is None or (i, j) not in tabu:
                    neighborhood.append((i, j))
        return neighborhood
        
    def best_neighbor_swap(self, tabu=None, threshold=0):
        # TODO: Implement this. The result should be the swap (i,j), when
        # applied to self, yields the best objective value. If no such (i,j)
        # exists, None should be returned.
        best_swap = None
        best_objective = float('inf') 
        for i in range(len(self.permutation)):
            for j in range(i + 1, len(self.permutation)):
                if tabu is None or (i, j) not in tabu:
                    new_permutation = self.permutation.copy()
                    new_permutation[i], new_permutation[j] = new_permutation[j], new_permutation[i]
                    new_objective = self.evaluate_objective(new_permutation, self.flows, self.distances)
                # search for a swap
                    if new_objective < best_objective:
                        best_objective = new_objective
                        best_swap = (i, j)
        
        return best_swap
        
    def swap(self, i, j):
        tmp = self.permutation[i]
        P = self.permutation.copy()
        P[i] = self.permutation[j]
        P[j] = tmp
        return Solution(P, self.flows, self.distances)

## Task 2: Implement Tabu Search

In [19]:
def tabu_search(initial_solution, tenure=10,budget=100):
    tabu = []
    X = initial_solution
    best_solution = initial_solution
    for iteration in range(1, budget + 1):
        print("Iteration {}".format(iteration))
        print("Current solution {}.".format(X))
        current_objective = X.objective
        best_swap = X.best_neighbor_swap(tabu, iteration)
        if best_swap is None: break
        X = X.swap(*best_swap)
        print("Best swap {} with objective {}. Difference to current {}.".format(best_swap, X.objective, X.objective - current_objective))
        if X.objective < best_solution.objective:
            best_solution = X
        print("Current best solution {} with objective {}.".format(best_solution, best_solution.objective))
        print("Tabu list state:")
        print(tabu)
        print()
    print("Best solution {} with objective {}.".format(best_solution, best_solution.objective))
    return best_solution

## Check solution for NUG5

In [20]:
nug5 = Solution.from_file("../res/data/nug5.dat")

In [21]:
# set an arbitrary permutation
sol = nug5
sol.permutation = np.array([2,4,1,5,3]) - 1

In [22]:
check(len(sol.neighbors())==10,"There are 10 neighbors")
check(sol.best_neighbor_swap()==(2,3),"(2,3) is the best initial swap")

There are 10 neighbors : passed.
(2,3) is the best initial swap : passed.


In [23]:
# Allow for a 10% deviation from optimal
acceptable_objective = nug5.optimal * 1.10
tenure = 5
budget = 500
np.random.seed(1)
random.seed(1)
sol = tabu_search(nug5, tenure=tenure, budget=budget)

Iteration 1
Current solution [1 3 0 4 2].
Best swap (2, 3) with objective 50.0. Difference to current -20.0.
Current best solution [1 3 4 0 2] with objective 50.0.
Tabu list state:
[]

Iteration 2
Current solution [1 3 4 0 2].
Best swap (0, 4) with objective 50.0. Difference to current 0.0.
Current best solution [1 3 4 0 2] with objective 50.0.
Tabu list state:
[]

Iteration 3
Current solution [2 3 4 0 1].
Best swap (0, 4) with objective 50.0. Difference to current 0.0.
Current best solution [1 3 4 0 2] with objective 50.0.
Tabu list state:
[]

Iteration 4
Current solution [1 3 4 0 2].
Best swap (0, 4) with objective 50.0. Difference to current 0.0.
Current best solution [1 3 4 0 2] with objective 50.0.
Tabu list state:
[]

Iteration 5
Current solution [2 3 4 0 1].
Best swap (0, 4) with objective 50.0. Difference to current 0.0.
Current best solution [1 3 4 0 2] with objective 50.0.
Tabu list state:
[]

Iteration 6
Current solution [1 3 4 0 2].
Best swap (0, 4) with objective 50.0. Dif

In [24]:
check(sol.objective<=nug5.optimal*1.10,f"Best objective should be within 10% of optimal {nug5.optimal}")

Best objective should be within 10% of optimal 50 : passed.


## Check solution for NUG20

In [25]:
nug20 = Solution.from_file("../res/data/nug20.dat")
# Set the tenure and budget for the search

tenure = int(0.20 * 20)
budget = 100
np.random.seed(1)
random.seed(1)
sol = tabu_search(nug20, tenure=tenure, budget=budget)

Iteration 1
Current solution [ 3 16  6 10  2 14  4 17  7  1 13  0 19 18  9 15  8 12 11  5].
Best swap (9, 11) with objective 3100.0. Difference to current -184.0.
Current best solution [ 3 16  6 10  2 14  4 17  7  0 13  1 19 18  9 15  8 12 11  5] with objective 3100.0.
Tabu list state:
[]

Iteration 2
Current solution [ 3 16  6 10  2 14  4 17  7  0 13  1 19 18  9 15  8 12 11  5].
Best swap (5, 7) with objective 2976.0. Difference to current -124.0.
Current best solution [ 3 16  6 10  2 17  4 14  7  0 13  1 19 18  9 15  8 12 11  5] with objective 2976.0.
Tabu list state:
[]

Iteration 3
Current solution [ 3 16  6 10  2 17  4 14  7  0 13  1 19 18  9 15  8 12 11  5].
Best swap (6, 18) with objective 2876.0. Difference to current -100.0.
Current best solution [ 3 16  6 10  2 17 11 14  7  0 13  1 19 18  9 15  8 12  4  5] with objective 2876.0.
Tabu list state:
[]

Iteration 4
Current solution [ 3 16  6 10  2 17 11 14  7  0 13  1 19 18  9 15  8 12  4  5].
Best swap (3, 5) with objective 2792

In [26]:
# Allow for a 10% deviation from optimal
check(sol.objective<=nug20.optimal*1.10,f"Best objective should be within 10% of optimal {nug20.optimal}")

Best objective should be within 10% of optimal 2570 : passed.


In [27]:
# Set the tenure, budget and the number of replicas for the search
replicas = 10

In [28]:
for i in range(replicas):
    np.random.seed(i)
    random.seed(i)
    nug20 = Solution.from_file("../res/data/nug20.dat")
    sol = tabu_search(nug20, tenure=tenure, budget=budget)
    # Allow for a 10% deviation from optimal
    check(sol.objective<=nug20.optimal*1.10,f"Best objective should be within 10% of optimal {nug20.optimal}")

Iteration 1
Current solution [18  1 19  8 10 17  6 13  4  2  5 14  9  7 16 11  3  0 15 12].
Best swap (4, 10) with objective 3144.0. Difference to current -256.0.
Current best solution [18  1 19  8  5 17  6 13  4  2 10 14  9  7 16 11  3  0 15 12] with objective 3144.0.
Tabu list state:
[]

Iteration 2
Current solution [18  1 19  8  5 17  6 13  4  2 10 14  9  7 16 11  3  0 15 12].
Best swap (2, 12) with objective 3052.0. Difference to current -92.0.
Current best solution [18  1  9  8  5 17  6 13  4  2 10 14 19  7 16 11  3  0 15 12] with objective 3052.0.
Tabu list state:
[]

Iteration 3
Current solution [18  1  9  8  5 17  6 13  4  2 10 14 19  7 16 11  3  0 15 12].
Best swap (6, 16) with objective 2980.0. Difference to current -72.0.
Current best solution [18  1  9  8  5 17  3 13  4  2 10 14 19  7 16 11  6  0 15 12] with objective 2980.0.
Tabu list state:
[]

Iteration 4
Current solution [18  1  9  8  5 17  3 13  4  2 10 14 19  7 16 11  6  0 15 12].
Best swap (15, 18) with objective 291